# 基于寒武纪MLU的模型训练--BERT 的 SQuAD 任务
### -- PyTorch，python，混合精度

## 目录
### 0 基本信息
### 1 实验内容及目标
     1.1 实验内容
     1.2 实验目标
### 2 前置知识介绍
     2.1 寒武纪软硬件平台
     2.2 寒武纪PyTorch框架
### 3 网络详解
     3.1 网络结构
### 4 模型训练
     4.1 工程目录介绍
     4.2 工程准备
     4.3 移植修改
     4.4 训练
     4.5 精度验证
### 5 结语
     5.1 回顾重点步骤
     5.2 相关链接


# 0 基本信息

发布者：寒武纪

实验时长：120 分钟

语言：Python3

修改时间：2022-11-17

# 1 实验内容及目标
## 1.1 实验内容

&emsp;&emsp;本实验主要介绍基于寒武纪 MLU370 (简称 MLU ) AI 处理器与寒武纪 PyTorch 框架的 BERT（基于transformers v4.0.1）训练方法。在官方源码的基础上，进行简单移植和修改，使用MLU370 加速训练 BERT 算法模型，并介绍基于 MLU370 的混合精度训练方法。后续章节将会详细介绍移植过程。


## 1.2 实验目标

1. 掌握使用寒武纪 MLU370 和 PyTorch 框架进行混合精度训练的基本方法。

2. 理解 BERT 模型的整体网络结构及其适配流程。


# 2 前置知识介绍

## 2.1 寒武纪软硬件平台介绍

 &emsp; 硬件：寒武纪 MLU370 AI 计算卡 
 
 &emsp; AI框架：PyTorch 1.6
 
 &emsp; 系统环境：寒武纪云平台 

## 2.2 寒武纪 PyTorch 框架

为⽀持寒武纪 MLU 加速卡，寒武纪定制了开源⼈⼯智能编程框架PyTorch（以下简称 Cambricon PyTorch）。
 
Cambricon PyTorch 借助 PyTorch ⾃⾝提供的设备扩展接⼝将 MLU 后端库中所包含的算⼦操作动态注册到 PyTorch 中，MLU 后端库可处理 MLU 上的张量和 AI 算⼦的运算。Cambricon PyTorch 会基于 CNNL 库在 MLU 后端实现⼀些常⽤AI 算⼦，并完成⼀些数据拷⻉。
 
Cambricon PyTorch 兼容原⽣ PyTorch 的 Python 编程接⼝和原⽣ PyTorch ⽹络模型，⽀持以在线逐层⽅式进⾏训练和推理。⽹络可以从模型⽂件中读取，对于训练任务，⽀持 float32，float16等混合精度。
获取更多有关Cambricon PTtorch资料，请参考[寒武纪官网文档](https://developer.cambricon.com/index/document/index/classid/3.html) PyTorch相关内容。

# 3 模型架构
## 3.1 网络结构
&emsp;&emsp;BERT 的全称为 Bidirectional Encoder Representation from Transformers，是一个预训练的语言表征模型, BERT 的每一层由一个 Encoder 单元构成。根据 Encoder 集成多少分为两种：
* 在比较大的BERT模型中，即BERT_large，有24层Encoder，每层中有16个Attention，词向量的维度是1024  
* 在比较小的BERT模型中，即BERT_base，有12层Encoder，每层有12个Attention，词向量维度是768  
BERT可以有效改进许多自然语言处理任务，包括自然语言分类，文本匹配，命名实体识别和SQuAD 问答等任务。本文主要介绍基于 BERT_base 的 SQuAD 问答任务，以下是其训练框架结构：   

<img src="course_images/bert_base_model.png" width="70%">  

其中      

* input 表示输入 Encoder之前的各 embedding 融合，包含 position embedding，segment embedding 和 token embedding;   

* Encoder 是 Transformer 的 Encoder 部分，每个Encoder 单元由一个 multi-head-Attention + Layer Normalization + feedforword + Layer Normalization 叠加产生      

# 4 模型训练

## 4.1 工程目录介绍
```
practices                              #  根目录    
├── LICENSE
├── README.md
├── model
    ├── pretrained
        ├── pretrain_bert_base_cased                         
├── dataset
    ├── private
        ├── squad                          
├── projects
    ├── caffe_ssd_inference               
    ├── pytorch_yolov5_inference         
    ├── pytorch_yolov5_train
    ├── pytorch_bert_train             #本实验工程目录
    ...

pytorch_bert_train/              
├── adaptation.sh                     # 适配MLU的脚本           
├── bert_amp.patch                    # 转换成使用 amp 的脚本
├── course_images                     # 本教程图片
│   └── bert_base_model.png
├── install_transformers.sh           # 安装 transformers 的脚本
├── output                            # 存放输出日志和模型
├── prepare.sh                        # 数据和模型准备
├── pretrain_bert_base_cased          # 预训练模型文件夹
├── pytorch_bert_train.ipynb          # 本实验案例
├── README.md
├── requirements.txt                  # 在容器环境外的其他依赖
└── transformers                      # v4.0.1的源码
    ├── CODE_OF_CONDUCT.md
    ├── CONTRIBUTING.md
    ├── docker
    ├── ...
```

## 4.2 工程准备


1. 安装依赖

In [ ]:
!pip install -r ./requirements.txt   

2. 下载原始 transformers 工程，切换到v4.0.1版本
```
git clone https://github.com/huggingface/transformers.git
git checkout v4.0.1
```

该步骤已经提前下载并切换了版本。

In [ ]:
# 并安装对应版本 transformers
!sh install_transformers.sh

3. 数据集和与预训练模型下载

默认使用 squad 数据集进行训练,预训练模型来自 huggingface 官网。下载后的存放路径如下：
```
/workspace \
|-- dataset                          
    |-- private
        |-- squad                          # 数据文件
        ...
|-- model                     
    |-- pretrained
        |--pretrain_bert_base_cased        # 模型文件    
      
```

In [ ]:
# 使用脚本下载预训练模型和训练及验证数据
!sh prepare.sh

## 4.3 移植修改
1. 脚本移植  

为了快捷地移植 PyTorch 代码，适配寒武纪 MLU 环境运行模型，寒武纪开发了对应的转换脚本，在安装了 Cambricon PyTorch 环境内均可使用，具体文件路径和名称为： /torch/src/catch/tools/torch_gpu2mlu.py，用户只需要使用脚本对所需转换的代码进行处理即可。

2. 混合精度训练  

&emsp;&emsp;Cambricon PyTorch ⽀持调⽤原⽣接⼝ torch.cuda.amp 实现⾃动混合精度训练，同时支持  torch.mlu.amp，效果相同 ，能够实现部分算⼦以float 类型计算，部分算⼦以 half 类型计算。
autocast 实例可通过上下⽂管理器或者装饰器对所在作⽤域下的算⼦实现混合精度训练。  
```
torch.mlu.amp.autocast(enabled=True)
```

&emsp;&emsp;custom_fwd 实例可以作为装饰器装饰 torch.autograd.Function ⼦类的 forward ⽅法，custom_bwd 实例则装饰该⼦类的 backward ⽅法，实现混合精度训练。
```
torch.mlu.amp.custom_fwd(fwd=None, **kwargs)
torch.mlu.amp.custom_bwd(bwd)
```

&emsp;&emsp;half 计算的算⼦可能会导致梯度下溢，因此 GradScaler 实例通过让 loss 乘上⽐例因⼦后进⾏反向传播，所得梯度再除以⽐例因⼦。为防⽌梯度上溢，需要检查梯度是否溢出。如果出现溢出，则不进⾏梯度更新。
```
torch.mlu.amp.GradScaler(init_scale=65536.0, growth_factor=2.0, backoff_factor=0.5, growth_int, erval=2000, enabled=True)
```

3. 使用转换脚本适配修改

In [ ]:
!sh adaptation.sh

1）使用其中脚本  python /torch/src/catch/tools/torch_gpu2mlu.py --input ./transformers/examples/question-answering/，可以在 ./transformers/examples/ 目录下生成适配于MLU的相关文件question-answering_mlu。  

2）其中 patch ./transformers/examples/question-answering_mlu/run_squad.py < bert_amp.patch 用于插入amp代码，用于进行混合进度训练。添加修改的主要内容如下：

```
# 初始化GradScaler
scaler = torch.mlu.amp.GradScaler()

# autocast实例通过上下⽂管理器进⾏混合精度训练
with torch.mlu.amp.autocast():
    outputs = model(**inputs)
    loss = outputs[0]
                                       
# 通过⽐例因⼦来缩放loss，随后进⾏反向传播                    
scaler.scale(loss).backward()   

# 梯度除以⽐例因⼦。如果没有梯度溢出，则更新梯度                    
scaler.step(optimizer)

# 更新⽐例因⼦
scaler.update()  
```

当选择使用混合精度训练时，会在原来fp32训练的基础上增加运行上述代码，达到混合精度训练效果。

## 4.4 训练

这里提供了两种运行方式：

1. From pretrained training：基于原始代码权重文件进行训练；
2. Resume Training：在上次训练基础上继续训练。

* 1. From pretrained training

In [ ]:
import os 
os.environ['MLU_VISIBLE_DEVICES']="0"
## 设置使用环境

In [ ]:
# Train BERT on squad for 1 epochs in mlu device
! python ./transformers/examples/question-answering_mlu/run_squad.py \
  --model_type bert \
  --model_name_or_path ./pretrain_bert_base_cased \
  --do_train \
  --fp16 \
  --do_lower_case \
  --train_file ./squad/train-v1.1.json \
  --predict_file ./squad/dev-v1.1.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 1.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --logging_steps 2000 \
  --save_steps 2000 \
  --overwrite_output_dir \
  --output_dir ./output

* model_type: 表示从 transformers 中选择模型，比如这里是bert，还可以选择"xlm", "roberta", "distilbert", "camembert", "bart", "longformer"等
* model_name_or_path： 预训练模型路径
* do_train：是否进行训练，设置表示进行训练
* do_eval：是否进行验证，设置表示进行验证
* fp16：是否启用混合精度，设置表示启用
* do_lower_case：若模型不能区分大小写，需要启用
* train_file：训练数据路径，本文是 ./squad/train-v1.1.json
* predict_file：验证数据路径：本文是 ./squad/dev-v1.1.json
* per_gpu_train_batch_size：训练时的batch的大小，本文设置为 12
* learning_rate：学习率，本文设置为 3e-5
* num_train_epochs：训练迭代次数，本文作为演示，设置为 1.0
* max_seq_length：最大总输入序列长度，长于此值的序列将被截断，短于此值的序列将被填充。本文设置为 384
* doc_stride：每次滑动的序列长度，这里设置为128
* logging_steps：迭代多久更新日志，这里设置为2000
* save_steps 2000：迭代多久保存模型文件，这里设置为2000
* overwrite_output_dir：覆盖输出目录的内容，启用表示覆盖
* output_dir：文件模型输出的路径

更多参数设置及解析在run_squad.py文件 parser.add_argument 中查看。

* 2. Resume Training

和1的训练基本一致，只需要将 model_name_or_path 对应的参数更换成需要最新模型所在的文件夹即可。

In [ ]:
! python ./transformers/examples/question-answering_mlu/run_squad.py \
  --model_type bert \
  --model_name_or_path ./output/checkpoint-6000 \
  --do_train \
  --do_eval \
  --fp16 \
  --do_lower_case \
  --train_file ./squad/train-v1.1.json \
  --predict_file ./squad/dev-v1.1.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 1.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --logging_steps 2000 \
  --save_steps 2000 \
  --overwrite_output_dir \
  --output_dir ./output

# 脚本参数解释同上

## 4.5 精度验证

In [ ]:
! python ./transformers/examples/question-answering_mlu/run_squad.py \
  --model_type bert \
  --model_name_or_path ./output/checkpoint-6000 \
  --do_eval \
  --fp16 \
  --do_lower_case \
  --predict_file ./squad/dev-v1.1.json \
  --max_seq_length 384 \
  --doc_stride 128 \
  --overwrite_output_dir \
  --output_dir ./output

# 脚本参数解释同上, 测试时注意根据上述训练权重保存情况修改 weight 的参数。

# 5 结语  

从上述适配流程可知，采用 MLU370 进行 AI 模型训练流程与其它 GPU 使用较为一致，方便用户学习与使用，极大降低了模型迁移成本和学习时间。同时采用 MLU370 训练能够加速模型的训练速度，与寒武纪MagicMind推理平台相配合使用，丰富训推一体平台的功能完善，让机器更好地理解与服务人类。更多有关于寒武纪训练和推理平台使用。

## 5.1 回顾重点步骤  
至此，基于寒武纪 MLU370 与 PyTorch 框架下的 BERT 训练实验已经完毕。让我们回顾一下主要开发步骤：
1. 新增MLU device支持，将模型与数据使用MLU进行训练；
2. 各种训练方式的使用，如采用finetune、resume以及自动混合精度的训练设置
3. 使用MLU进行精度验证；

## 5.2 相关链接  

对上述代码有疑问请提交ISSUE:    
https://gitee.com/cambricon/practices/issues    

更多与寒武纪开发相关的有趣内容请移步至寒武纪开发者社区：    
https://developer.cambricon.com/    

如果有任何其他问题可到寒武纪开发者论坛提问，会有专人为您解答：    
https://forum.cambricon.com//list-1-1.html   
